In [1]:
import numpy as np
import pandas as pd

In [17]:
df = pd.read_csv('resources/train.csv')

In [20]:
df.sort_values('user_id').head()

,user_id,restaurant_id,type,timestamp
713262,1,3552,bookmark,3/4/2014
713258,1,7,bookmark,3/4/2014
713259,1,3622,bookmark,3/4/2014
713260,1,5649,bookmark,3/4/2014
713261,1,1873,bookmark,3/4/2014


In [23]:
temp = df.groupby(['restaurant_id']).size()
drop_index = temp.loc[temp.values > 50].index
df = df.loc[df['restaurant_id'].isin(drop_index)]

In [24]:
drop_index

Int64Index([     2,      4,      6,      7,      9,     14,     18,     20,
                21,     27,
            ...
            244276, 246384, 249722, 253792, 254296, 258714, 271867, 274507,
            281503, 377124],
           dtype='int64', name='restaurant_id', length=3368)

In [25]:
df['score'] = df.apply(lambda row: 2 if row['type']=='check-in' else 1 , axis = 1)

In [26]:
train_data = df.drop(columns=['type','timestamp'])

In [27]:
def calculate_score(df):
    return df['score'].sum()

In [28]:
temp = train_data.groupby(['restaurant_id','user_id']).apply(calculate_score)

In [29]:
temp = pd.DataFrame(temp).reset_index()

In [30]:
temp.columns = ['restaurant_id', 'user_id', 'score']

In [31]:
temp.to_csv(path_or_buf='resources/filtered_train.csv', index=False)